<a href="https://colab.research.google.com/github/mr-alamdari/NLP-Parts-of-Speech-Tagging-Begginer/blob/main/NLP_Parts_of_Speech_Tagging_Begginer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import os
import re
import collections
import numpy as np
import pandas as pd

In [3]:
!wget https://raw.githubusercontent.com/amanjeetsahu/Natural-Language-Processing-Specialization/master/Natural%20Language%20Processing%20with%20Probabilistic%20Models/Week%202/WSJ_24.pos

--2022-04-15 12:00:45--  https://raw.githubusercontent.com/amanjeetsahu/Natural-Language-Processing-Specialization/master/Natural%20Language%20Processing%20with%20Probabilistic%20Models/Week%202/WSJ_24.pos
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 286063 (279K) [text/plain]
Saving to: ‘WSJ_24.pos.1’

WSJ_24.pos.1        100%[===================>] 279.36K  --.-KB/s    in 0.02s   

2022-04-15 12:00:45 (12.4 MB/s) - ‘WSJ_24.pos.1’ saved [286063/286063]



In [4]:
!wget https://raw.githubusercontent.com/amanjeetsahu/Natural-Language-Processing-Specialization/master/Natural%20Language%20Processing%20with%20Probabilistic%20Models/Week%202/WSJ_02-21.pos

--2022-04-15 12:00:46--  https://raw.githubusercontent.com/amanjeetsahu/Natural-Language-Processing-Specialization/master/Natural%20Language%20Processing%20with%20Probabilistic%20Models/Week%202/WSJ_02-21.pos
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8279089 (7.9M) [text/plain]
Saving to: ‘WSJ_02-21.pos.1’

WSJ_02-21.pos.1     100%[===================>]   7.90M  --.-KB/s    in 0.07s   

2022-04-15 12:00:46 (115 MB/s) - ‘WSJ_02-21.pos.1’ saved [8279089/8279089]



In [5]:
!wget https://raw.githubusercontent.com/amanjeetsahu/Natural-Language-Processing-Specialization/master/Natural%20Language%20Processing%20with%20Probabilistic%20Models/Week%202/hmm_vocab.txt

--2022-04-15 12:03:04--  https://raw.githubusercontent.com/amanjeetsahu/Natural-Language-Processing-Specialization/master/Natural%20Language%20Processing%20with%20Probabilistic%20Models/Week%202/hmm_vocab.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 196571 (192K) [text/plain]
Saving to: ‘hmm_vocab.txt’

hmm_vocab.txt       100%[===================>] 191.96K  --.-KB/s    in 0.02s   

2022-04-15 12:03:04 (10.4 MB/s) - ‘hmm_vocab.txt’ saved [196571/196571]



In [6]:
with open('WSJ_02-21.pos', 'r') as f:
  train_corpus = f.readlines()

In [8]:
train_corpus[10: 20]

['at\tIN\n',
 'Chicago\tNNP\n',
 "'s\tPOS\n",
 'Goodman\tNNP\n',
 'Theatre\tNNP\n',
 '(\t(\n',
 '``\t``\n',
 'Revitalized\tVBN\n',
 'Classics\tNNS\n',
 'Take\tVBP\n']

In [9]:
with open('WSJ_24.pos', 'r') as f:
  y = f.readlines()

In [10]:
y[10: 20]

['points\tNNS\n',
 'this\tDT\n',
 'week\tNN\n',
 ',\t,\n',
 'with\tIN\n',
 'readings\tNNS\n',
 'on\tIN\n',
 'trade\tNN\n',
 ',\t,\n',
 'output\tNN\n']

In [20]:
with open('hmm_vocab.txt', 'r') as f:
  vocabulary_list = f.read().split('\n')

vocab = {}
for ind, word in enumerate(sorted(vocabulary_list)):
  vocab[word] = ind

In [21]:
sorted(vocabulary_list)[10: 20]

["'70s", "'80s", "'86", "'90s", "'N", "'S", "'d", "'em", "'ll", "'m"]

In [27]:
for id, (k, v) in enumerate(vocab.items()):
  print(k, v)
  if id == 10:
    break

 0
! 1
# 2
$ 3
% 4
& 5
' 6
'' 7
'40s 8
'60s 9
'70s 10


In [76]:
# https://github.com/amanjeetsahu/Natural-Language-Processing-Specialization/blob/master/Natural%20Language%20Processing%20with%20Probabilistic%20Models/Week%202/utils_pos.py

import string


# Punctuation characters
punct = set(string.punctuation)

# Morphology rules used to assign unknown word tokens
noun_suffix = ["action", "age", "ance", "cy", "dom", "ee", "ence", "er", "hood", "ion", "ism", "ist", "ity", "ling", "ment", "ness", "or", "ry", "scape", "ship", "ty"]
verb_suffix = ["ate", "ify", "ise", "ize"]
adj_suffix = ["able", "ese", "ful", "i", "ian", "ible", "ic", "ish", "ive", "less", "ly", "ous"]
adv_suffix = ["ward", "wards", "wise"]



def word_tag_spliter(word_tag, vocab):

  if word_tag.split():
    word, tag = word_tag.split()
    if not vocab.get(word):
      word = assign_unk(word)
  else:
      word, tag = '--n--', '--s--'
  
  return word, tag

def assign_unk(tok):
    """
    Assign unknown word tokens
    """
    # Digits
    if any(char.isdigit() for char in tok):
        return "--unk_digit--"

    # Punctuation
    elif any(char in punct for char in tok):
        return "--unk_punct--"

    # Upper-case
    elif any(char.isupper() for char in tok):
        return "--unk_upper--"

    # Nouns
    elif any(tok.endswith(suffix) for suffix in noun_suffix):
        return "--unk_noun--"

    # Verbs
    elif any(tok.endswith(suffix) for suffix in verb_suffix):
        return "--unk_verb--"

    # Adjectives
    elif any(tok.endswith(suffix) for suffix in adj_suffix):
        return "--unk_adj--"

    # Adverbs
    elif any(tok.endswith(suffix) for suffix in adv_suffix):
        return "--unk_adv--"

    return "--unk--"

def preprocess(vocab, data_fp):
    """
    Preprocess data
    """
    orig = []
    prep = []

    with open(data_fp, "r") as data_file:
        for cnt, word in enumerate(data_file):
            # End of sentence
            if not word.split():
                orig.append(word.strip())
                word = "--n--"
                prep.append(word)
                continue

            # Handle unknown words
            elif word.strip() not in vocab:
                orig.append(word.strip())
                word = assign_unk(word)
                prep.append(word)
                continue

            else:
                orig.append(word.strip())
                prep.append(word.strip())

    assert(len(orig) == len(open(data_fp, "r").readlines()))
    assert(len(prep) == len(open(data_fp, "r").readlines()))

    return orig, prep


In [29]:
!wget https://raw.githubusercontent.com/amanjeetsahu/Natural-Language-Processing-Specialization/master/Natural%20Language%20Processing%20with%20Probabilistic%20Models/Week%202/test.words

--2022-04-15 12:43:47--  https://raw.githubusercontent.com/amanjeetsahu/Natural-Language-Processing-Specialization/master/Natural%20Language%20Processing%20with%20Probabilistic%20Models/Week%202/test.words
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 180266 (176K) [text/plain]
Saving to: ‘test.words’

test.words          100%[===================>] 176.04K  --.-KB/s    in 0.01s   

2022-04-15 12:43:47 (11.6 MB/s) - ‘test.words’ saved [180266/180266]



In [77]:
_, prep = preprocess(vocab, 'test.words')

In [78]:
prep[10:20]

['points',
 'this',
 'week',
 ',',
 'with',
 'readings',
 'on',
 'trade',
 ',',
 'output']

In [79]:
from collections import defaultdict

def dictionary_creator(corpus, vocab):
  emission_counts = defaultdict(int)
  transition_counts = defaultdict(int)
  tag_counts = defaultdict(int)
  prev_tag = '--s--'
  for word_tag in corpus:
    word, tag = word_tag_spliter(word_tag, vocab)

    transition_counts[(prev_tag, tag)] += 1
    emission_counts[(tag, word)] += 1
    tag_counts[tag] += 1
    prev_tag = tag
  
  return emission_counts, transition_counts, tag_counts 

In [80]:
emission_counts, transition_counts, tag_counts = dictionary_creator(train_corpus, vocab)

In [90]:
for i, item in enumerate(emission_counts.items()):
  print(item[0], item[1])
  if i == 20:
    break

('IN', 'In') 1735
('DT', 'an') 3142
('NNP', 'Oct.') 317
('CD', '19') 100
('NN', 'review') 36
('IN', 'of') 22925
('``', '``') 6967
('DT', 'The') 6795
('NN', 'Misanthrope') 3
("''", "''") 6787
('IN', 'at') 4361
('NNP', 'Chicago') 197
('POS', "'s") 8079
('NNP', 'Goodman') 7
('NNP', 'Theatre') 5
('(', '(') 1153
('VBN', '--unk_upper--') 93
('NNS', '--unk_upper--') 350
('VBP', 'Take') 1
('DT', 'the') 41098
('NN', 'Stage') 1


In [91]:
for i, item in enumerate(transition_counts.items()):
  print(item[0], item[1])
  if i == 20:
    break

('--s--', 'IN') 5050
('IN', 'DT') 32364
('DT', 'NNP') 9044
('NNP', 'CD') 1752
('CD', 'NN') 7377
('NN', 'IN') 32885
('IN', '``') 546
('``', 'DT') 1014
('DT', 'NN') 38873
('NN', "''") 686
("''", 'IN') 591


In [92]:
for i, item in enumerate(tag_counts.items()):
  print(item[0], item[1])
  if i == 20:
    break

IN 98554
DT 81842
NNP 91466
CD 36568
NN 132935
`` 7092
'' 6919
POS 8701
( 1366
VBN 20024
NNS 59856
VBP 12491
, 48727
CC 23947
) 1376
VBD 29889
RB 30970
TO 22357
. 39478
--s-- 39832
VBZ 21672


In [121]:
def pos_predictor(prep, y, emission_counts, vocab, states): 
  corrects = 0
  total = len(y)

  for word, word_pos in zip(prep, y):
    
    if len(word_pos) == 1:
      continue

    true_word, true_pos = word_pos.split()
    final_count, final_pos = 0, ''
 
    if vocab.get(word):
      for pos in states:
        
        key = (pos, word)
        count = emission_counts.get(key)
        
        if count != None and count > final_count:
          final_count = count
          final_pos = pos
      
      if final_pos == true_pos:
        corrects += 1

  accuracy = corrects / total
  
  return accuracy

In [122]:
pos_predictor(prep, y, emission_counts, vocab, states)

Streaming output truncated to the last 5000 lines.
NN 101
DT 1168
IN 4815
MD 193
RB 3222
VB 3724
RB 17
VBD 5
IN 4483
PRP$ 1678
NNS 43
. 39020
DT 6795
NN 195
NNP 1
POS 8079
NN 194
VBD 3903
VBD 2
VBN 4
FW 2
IN 14957
DT 19264
$ 1
CD 4
JJ 1942
JJ 20
VBD 81
VBN 111
FW 2
IN 14957
CD 1220
IN 22925
CD 591
NN 101
NNS 139
DT 41098
NNP 82
NN 3
NNP 130
MD 3055
VB 27
VBD 34
CC 16113
MD 3055
VB 42
DT 1897
NN 88
. 39020
DT 6795
NN 195
VBD 81
VBN 111
DT 41098
JJ 20
FW 2
IN 14957
, 1
DT 3142
NN 39
VBG 58
DT 19264
NNP 14
NN 298
DT 1168
IN 4815
VBZ 34
DT 1168
IN 4815
CC 25
DT 336
NNS 43
IN 22925
DT 19264
JJ 51
VB 3724
VBD 3
VBN 10
IN 469
PRP 384
MD 808
JJ 1
VB 1108
VBP 1932
, 1
DT 3142
NN 101
. 39020
DT 6795
NN 195
VBD 31
DT 41098
NNS 3
TO 22198
VB 19
DT 19264
JJ 148
JJ 53
JJ 1
VBD 31
IN 4483
JJ 1
NNP 20
NN 3
NNP 80
NNP 126
CD 5
FW 12
IN 13
JJ 244
NN 365
NNP 4635
JJ 1007
NNP 193
WRB 987
DT 41098
JJ 315
NN 274
VBD 6
NNP 20
NNP 1
POS 8079
NN 101
NNS 64
. 39020
IN 136
DT 1168
IN 4815
JJ 53
, 48723
WDT 2218


0.8888563993099213

In [151]:
def transition_matrix(transition_counts, tag_counts, alpha):
  
  all_tags = sorted(tag_counts.keys())
  tags_count = len(all_tags)
  a = np.zeros((tags_count, tags_count))
  transition_krys = set(transition_counts.keys())

  for i in range(tags_count):
    for j in range(tags_count):
      key = (all_tags[i], all_tags[j])
      
      tr_count = transition_counts.get(key)

      count = tr_count if tr_count else 0

      prev_tag_count = tag_counts.get(all_tags[i])

      a[i, j] = (count+alpha) / (prev_tag_count + tags_count*alpha)
  
  return a

In [196]:
a = transition_matrix(transition_counts, tag_counts, 0.001)

In [197]:
a

array([[7.03997297e-06, 7.03997297e-06, 7.03997297e-06, ...,
        7.03997297e-06, 7.03997297e-06, 7.03997297e-06],
       [1.35647553e-07, 1.35647553e-07, 1.35647553e-07, ...,
        1.35647553e-07, 1.35647553e-07, 1.35647553e-07],
       [1.44528595e-07, 1.44673124e-04, 6.93751711e-03, ...,
        2.89201719e-04, 3.17977363e-03, 2.74618784e-03],
       ...,
       [5.95075158e-06, 2.38089571e-02, 5.95075158e-06, ...,
        5.95075158e-06, 5.95075158e-06, 5.95670233e-03],
       [4.66625541e-07, 4.67092167e-04, 4.66625541e-07, ...,
        4.66625541e-07, 4.67092167e-04, 1.86696879e-03],
       [1.41003034e-07, 1.41144037e-04, 1.41003034e-07, ...,
        1.41003034e-07, 1.12803837e-02, 4.23150104e-04]])

In [290]:
def emission_matrix(emission_counts, tag_counts, vocab, alpha):
  tags_count = len(tag_counts)
  all_tags = sorted(tag_counts.keys())
  words_count = len(vocab)
  emission_keys = set(list(emission_counts.keys()))
  b = np.zeros((tags_count, words_count))

  for i in range(tags_count):
    for j in range(words_count):
      count = 0
      key = (all_tags[i], vocab[j])
      if key in emission_counts.keys():
        count = emission_counts[key]

      count_tag = tag_counts[all_tags[i]]
      b[i, j] = (count + alpha) / (count_tag + alpha*words_count)
  return b

In [291]:
b = emission_matrix(emission_counts, tag_counts, list(vocab), 0.001)

In [293]:
b

array([[6.03219988e-06, 6.03219988e-06, 8.56578416e-01, ...,
        6.03219988e-06, 6.03219988e-06, 6.03219988e-06],
       [1.35212298e-07, 1.35212298e-07, 1.35212298e-07, ...,
        1.35212298e-07, 1.35212298e-07, 1.35212298e-07],
       [1.44034584e-07, 1.44034584e-07, 1.44034584e-07, ...,
        1.44034584e-07, 1.44034584e-07, 1.44034584e-07],
       ...,
       [5.21438963e-06, 5.21438963e-06, 5.21438963e-06, ...,
        5.21438963e-06, 5.21438963e-06, 5.21438963e-06],
       [4.61514960e-07, 4.61514960e-07, 4.61514960e-07, ...,
        4.61514960e-07, 4.61514960e-07, 4.61514960e-07],
       [1.40532791e-07, 1.40532791e-07, 1.40532791e-07, ...,
        1.40532791e-07, 1.40532791e-07, 1.40532791e-07]])

In [294]:
def initialization(states, tag_counts, a, b, corpus, vocab):
  tags_count = len(tag_counts)
  best_probs = np.zeros((tags_count, len(corpus)))
  best_paths = np.zeros((tags_count, len(corpus)), dtype=int)
  s_ind = states.index('--s--')

  for i in range(tags_count):
    best_probs[i, 0] = float('-inf') if a[s_ind, i]==0 else np.log(a[s_ind,i]) + np.log(b[i,vocab[corpus[0]]])

  return best_probs, best_paths

In [295]:
best_probs, best_paths = initialization(states, tag_counts, a, b, prep, vocab)

In [296]:
best_probs

array([[-22.60982633,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [-23.07660654,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [-23.57298822,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       ...,
       [-22.75551606,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [-19.6637215 ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [-18.36288463,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ]])

In [297]:
best_paths

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [298]:
def viterbi_forward(a, b, corpus, best_probs, best_paths, vocab):
  
  tags_count = best_probs.shape[0]

  for i in range(1, len(corpus)):
    for j in range(tags_count):
      best_prob_i = float('-inf')
      best_path_i = None

      for k in range(tags_count):
        prob = best_probs[k, i-1] + np.log(a[k, j]) + np.log(b[j, vocab.get(corpus[i])])
        if prob > best_prob_i:
          best_prob_i = prob
          best_path_i = k
        
      best_probs[j, i] = best_prob_i
      best_paths[j, i] = best_path_i

  return best_probs, best_paths

In [299]:
def viterbi_forward(A, B, test_corpus, best_probs, best_paths, vocab):
    '''
    Input: 
        A, B: The transiton and emission matrices respectively
        test_corpus: a list containing a preprocessed corpus
        best_probs: an initilized matrix of dimension (num_tags, len(corpus))
        best_paths: an initilized matrix of dimension (num_tags, len(corpus))
        vocab: a dictionary where keys are words in vocabulary and value is an index 
    Output: 
        best_probs: a completed matrix of dimension (num_tags, len(corpus))
        best_paths: a completed matrix of dimension (num_tags, len(corpus))
    '''
    # Get the number of unique POS tags (which is the num of rows in best_probs)
    num_tags = best_probs.shape[0]
    
    # Go through every word in the corpus starting from word 1
    # Recall that word 0 was initialized in `initialize()`
    for i in range(1, len(test_corpus)): 
        
        # Print number of words processed, every 5000 words
        if i % 5000 == 0:
            print("Words processed: {:>8}".format(i))
            
        ### START CODE HERE (Replace instances of 'None' with your code EXCEPT the first 'best_path_i = None') ###
        # For each unique POS tag that the current word can be
        for j in range(num_tags): # complete this line
            
            # Initialize best_prob for word i to negative infinity
            best_prob_i = float("-inf")
            
            # Initialize best_path for current word i to None
            best_path_i = None

            # For each POS tag that the previous word can be:
            for k in range(num_tags): # complete this line
            
                # Calculate the probability = 
                # best probs of POS tag k, previous word i-1 + 
                # log(prob of transition from POS k to POS j) + 
                # log(prob that emission of POS j is word i)
                prob = best_probs[k,i-1]+np.log(A[k,j]) +np.log(B[j,vocab[test_corpus[i]]])

                # check if this path's probability is greater than
                # the best probability up to and before this point
                if prob > best_prob_i: # complete this line
                    
                    # Keep track of the best probability
                    best_prob_i = prob
                    
                    # keep track of the POS tag of the previous word
                    # that is part of the best path.  
                    # Save the index (integer) associated with 
                    # that previous word's POS tag
                    best_path_i = k

            # Save the best probability for the 
            # given current word's POS tag
            # and the position of the current word inside the corpus
            best_probs[j,i] = best_prob_i
            
            # Save the unique integer ID of the previous POS tag
            # into best_paths matrix, for the POS tag of the current word
            # and the position of the current word inside the corpus.
            best_paths[j,i] = best_path_i

        ### END CODE HERE ###
    return best_probs, best_paths

In [300]:
best_probs, best_paths = viterbi_forward(a, b, prep, best_probs, best_paths, vocab)

Words processed:     5000
Words processed:    10000
Words processed:    15000
Words processed:    20000
Words processed:    25000
Words processed:    30000


In [301]:
best_probs

array([[-2.26098263e+01, -2.47821563e+01, -3.40824650e+01, ...,
        -2.08553710e+05, -2.08571000e+05, -2.08575297e+05],
       [-2.30766065e+01, -2.45158390e+01, -3.50477430e+01, ...,
        -2.08553663e+05, -2.08566281e+05, -2.08579095e+05],
       [-2.35729882e+01, -2.99830506e+01, -3.19800466e+01, ...,
        -2.08559220e+05, -2.08565630e+05, -2.08564366e+05],
       ...,
       [-2.27555161e+01, -3.44006276e+01, -3.17437063e+01, ...,
        -2.08557240e+05, -2.08571146e+05, -2.08575443e+05],
       [-1.96637215e+01, -2.99165357e+01, -3.15812704e+01, ...,
        -2.08555283e+05, -2.08563618e+05, -2.08570959e+05],
       [-1.83628846e+01, -2.49885094e+01, -3.27766333e+01, ...,
        -2.08555245e+05, -2.08563741e+05, -2.08569258e+05]])

In [302]:
best_paths
print(f"best_probs[0,1]: {best_probs[0,1]:.4f}") 
print(f"best_probs[0,4]: {best_probs[0,4]:.4f}") 

best_probs[0,1]: -24.7822
best_probs[0,4]: -49.5601


In [303]:
def viterbi_backward(best_probs, best_path, corpus, states):
  m = best_paths.shape[1]
  tags_count = best_paths.shape[0]
  last_word_best_prob =float('-inf')
  z = [None] * m
  pred = [None] * m

  for k in range(tags_count):
    if best_probs[k, -1]>last_word_best_prob:
      last_word_best_prob = best_probs[k, -1]
      z[m-1] = k
  
  pred[m-1] = states[k]
  for i in range(len(corpus)-1, -1, -1):
    pos_tag_for_word_i = best_paths[np.argmax(best_probs[:, i]), i]
    z[i-1] = best_paths[pos_tag_for_word_i, i]
    pred[i-1] = states[pos_tag_for_word_i]
  return pred

In [304]:
pred = viterbi_backward(best_probs, best_paths, prep, states)

In [305]:
# pred[10:20]
m = len(pred)
print('The prediction for pred[-7:m-1] is: \n', prep[-7:m-1], "\n", pred[-7:m-1], "\n")
print('The prediction for pred[0:8] is: \n', pred[0:7], "\n", prep[0:7])


The prediction for pred[-7:m-1] is: 
 ['see', 'them', 'here', 'with', 'us', '.'] 
 ['VB', 'PRP', 'RB', 'IN', 'PRP', '.'] 

The prediction for pred[0:8] is: 
 ['DT', 'NN', 'POS', 'NN', 'MD', 'VB', 'VBN'] 
 ['The', 'economy', "'s", 'temperature', 'will', 'be', 'taken']


In [306]:
def accuracy(pred, y):
  corrects = 0

  for p, y_id in zip(pred, y):
    word_tag = y_id.split()
    if len(word_tag) != 2:
      continue

    true_word, true_tag = word_tag
    
    if p == true_tag:
      corrects += 1
  return corrects / len(y)

In [307]:
accuracy(pred, y)

0.9152606801368461